In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-05-1"
start_date = "2000-01-01"

In [3]:
def getGOLDData ():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last']]
    data.columns = ["GOLD Adj Close"]
    return data

def getSPData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["SP500 Adj Close",  "SP500 Volume"]
    return data

def getDAXData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:6]]
    data.columns = ["DAX Adj Close",  "DAX Volume"]
    return data


def getOILData():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last"]]
    data.columns=["OIL Adj Close"]
    return data


def getNIKKEIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["FTSE Adj Close",  "FTSE Volume"]
    return data

def getSHANGHAIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [4]:
def checkData (data):
    counter = 0
    for index, row in data.iterrows():
        for item in row:
            if (math.isnan(item)):
                counter += 1
                break
    return counter

def normalizeData(data):
    for column in data:
        maxValue = max(data[column])
        data[column] = data[column] / maxValue

In [5]:
def combineData():
    #allData = [getSPData(), getGOLDData(), getDAXData(), getOILData(), getNIKKEIData(), getSHANGHAIData(), getFTSEData()]
    #allData = [getSPData()]
    allData = [getSPData(), getDAXData(), getFTSEData(), getGOLDData(), getOILData()]
    mergedData = pd.concat(allData, axis = 1)
    print("REMOVED: {} DATA POINTS".format(checkData(mergedData)))
    cleanData = mergedData.dropna()
    #normalizeData(cleanData)
    return cleanData

In [6]:
data = combineData()

print(data)

day_counter = 0

# Size of prediction
prediction_size = 1

# Number of days in the past
historical_size = 15

# Size of available data.
data_size = len(data) - prediction_size - historical_size

# Size of data alocated to training
training_size = int(0.85*data_size)

# Size of data alocated to testing
testing_size = data_size - training_size

# Number of data elements in a single day
daily_data_size = 8 

x_train = np.zeros((training_size, historical_size, daily_data_size))
y_train = np.zeros((training_size))
x_test  = np.zeros((testing_size, historical_size, daily_data_size))
y_test  = np.zeros((testing_size))

sanity_train = np.zeros((training_size, 3))
sanity_test = np.zeros((testing_size, 3))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
REMOVED: 339 DATA POINTS
            SP500 Adj Close  SP500 Volume  DAX Adj Close   DAX Volume  \
Date                                                                    
2000-01-04      1399.420044  1.009000e+09    6586.950195   46678400.0   
2000-01-05      1402.109985  1.085500e+09    6502.069824   52682800.0   
2000-01-06      1403.449951  1.092300e+09    6474.919922   41180600.0   
2000-01-07      1441.469971  1.225200e+09    6780.959961   56058900.0   
2000-01-10      1457.599976  1.064800e+09    6925.520020   42006200.0   
...                     ...           ...            ...          ...   
2020-04-24      2836.739990  5.374480e+09   10336.089844  120035000.0   
2020-04-27      2878.479980  5.194260e+09   10659.990234  122476600.0   
2020-04-28      2863.389893  5.672880e

In [7]:
day_counter = 0

for index, row in data.iterrows():
    
    if (day_counter == data_size):
        print("100% completed")
        break

    if (day_counter < len(x_train)):
        
        
        for i in range (historical_size):
            for j in range (daily_data_size):
                x_train[day_counter][i][j] = data.iloc[day_counter + i][j]
                
        base_value = data.iloc[day_counter + historical_size - 1]['SP500 Adj Close']
        future_values = []
        for i in range (prediction_size):
            future_values += [data.iloc[day_counter + historical_size + i]['SP500 Adj Close']] 
        average_future = sum(future_values) / len(future_values)
        if ((average_future - base_value) > 0):
            delta = 1
        else:
            delta = 0
            
        y_train[day_counter] = delta
        
        sanity_train[day_counter] = np.array(future_values)
        
        
    else:
        
        for i in range (historical_size):
            for j in range (daily_data_size):
                x_test[day_counter - len(x_train)][i][j] = data.iloc[day_counter + i][j]
                
        base_value = data.iloc[day_counter + historical_size - 1]['SP500 Adj Close']
        future_values = []
        for i in range (prediction_size):
            future_values += [data.iloc[day_counter + historical_size + i]['SP500 Adj Close']] 
        average_future = sum(future_values) / len(future_values)
        if ((average_future - base_value) > 0):
            delta = 1
        else:
            delta = 0
            
        y_test[day_counter - len(y_train)] = delta
        
        sanity_test[day_counter - len(y_train)] = np.array(future_values)
        
    day_counter += 1
    print("{}% completed".format(int(100 * day_counter/data_size)), end = '\r')

100% completed


In [8]:
y_train = y_train.astype('uint8')
y_test  = y_test.astype('uint8') 

In [9]:
# Balance out training data
ones = 0
zeros = 0
for i in y_train:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
balanced_x_train = np.zeros((2*min(ones,zeros), historical_size, daily_data_size))
balanced_y_train = np.zeros((2*min(ones,zeros)))
balanced_sanity_train = np.zeros((2*min(ones,zeros), 3))

# Insert ones
index = 0
for i in range (len(x_train)):
    if (y_train[i] == 1):
        balanced_x_train[index] = x_train[i]
        balanced_y_train[index] = y_train[i]
        balanced_sanity_train[index] = sanity_train[i]
        index += 1
        
    if (index == min(ones, zeros)):
        break
        
# Insert zeros
for i in range (len(x_train)):
    if (y_train[i] == 0):
        balanced_x_train[index] = x_train[i]
        balanced_y_train[index] = y_train[i]
        balanced_sanity_train[index] = sanity_train[i]
        index += 1
        
    if (index == len(balanced_x_train)):
        break

In [10]:
# Balance out testing data
ones = 0
zeros = 0
for i in y_test:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
balanced_x_test = np.zeros((2*min(ones,zeros), historical_size, daily_data_size))
balanced_y_test = np.zeros((2*min(ones,zeros)))
balanced_sanity_test = np.zeros((2*min(ones,zeros), 3))

# Insert ones
index = 0
for i in range (len(x_test)):
    if (y_test[i] == 1):
        balanced_x_test[index] = x_test[i]
        balanced_y_test[index] = y_test[i]
        balanced_sanity_test[index] = sanity_test[i]
        index += 1
        
    if (index == min(ones, zeros)):
        break
        
# Insert zeros
for i in range (len(x_test)):
    if (y_test[i] == 0):
        balanced_x_test[index] = x_test[i]
        balanced_y_test[index] = y_test[i]
        balanced_sanity_test[index] = sanity_test[i]
        index += 1
        
    if (index == len(balanced_x_test)):
        break

In [11]:
ones = 0
zeros = 0
for i in balanced_y_test:
    if (i == 1):
        ones += 1
    else:
        zeros += 1
        
print(ones)
print(zeros)

322
322


In [12]:
# Shuffle training data
randomize_training = np.arange(len(balanced_x_train))
np.random.shuffle(randomize_training)
balanced_x_train = balanced_x_train[[randomize_training]]
balanced_y_train = balanced_y_train[[randomize_training]]
balanced_sanity_train = balanced_sanity_train[[randomize_training]]

# Shuffle testing data
randomize_testing = np.arange(len(balanced_x_test))
np.random.shuffle(randomize_testing)
balanced_x_test = balanced_x_test[[randomize_testing]]
balanced_y_test = balanced_y_test[[randomize_testing]]
balanced_sanity_test = balanced_sanity_test[[randomize_testing]]

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of

In [13]:
balanced_y_train = balanced_y_train.astype('uint8')
balanced_y_test  = balanced_y_test.astype('uint8') 

In [14]:
# Normalize training data

def find_max_in_col(df, column):
    data = []
    for sequence in df:
        for day in sequence:
            data.append(day[column])
    return max(data)

# 1 - Find the maximums of each column
maximums = []
for daily_feature in range (daily_data_size):
    maximums += [find_max_in_col(balanced_x_train, daily_feature)]
    
# 2 - Divide each number by the maximum
for sequence in range (len(balanced_x_train)):
    for day in range (historical_size):
        for daily_feature in range (daily_data_size):
            balanced_x_train[sequence][day][daily_feature] = balanced_x_train[sequence][day][daily_feature] / maximums[daily_feature]

In [15]:
# Normalize testing data

def find_max_in_col(df, column):
    data = []
    for sequence in df:
        for day in sequence:
            data.append(day[column])
    return max(data)

# 1 - Find the maximums of each column
maximums = []
for daily_feature in range (daily_data_size):
    maximums += [find_max_in_col(balanced_x_test, daily_feature)]
    
# 2 - Divide each number by the maximum
for sequence in range (len(balanced_x_test)):
    for day in range (historical_size):
        for daily_feature in range (daily_data_size):
            balanced_x_test[sequence][day][daily_feature] = balanced_x_test[sequence][day][daily_feature] / maximums[daily_feature]

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
representation_spaces = [32,64,128,256]
drop_out_multipliers = [1/2, 3/4, 1, 5/4, 2]
optimizers = ['Adam', 'SGD']
learning_rates = [0.00005, 0.0001, 0.0005,0.001]
batch_sizes = [32, 64, 128] 

# Maps model names to their validation accuracy
results = {}

counter = 1
for representation_space in representation_spaces:
    for drop_out_multiplier in drop_out_multipliers:
        for optimizer in optimizers:
            for learning_rate in learning_rates:
                for batch_size in batch_sizes:
                
                    NAME = "representation_space({})-drop_out_multiplier({})-optimizer({})-learning_rate({})-batch_size({})".format( \
                        representation_space,drop_out_multiplier, optimizer, learning_rate, batch_size)
                    
                    print("{} / {} models".format(counter, 
len(drop_out_multipliers) * len(optimizers) * len(learning_rates) * len(batch_sizes) * len(representation_spaces)))
                    print("Model: {}".format(NAME))
                    
                    print("-------------------------------------------")
                    
                    model = Sequential()

                    model.add(LSTM(representation_space, input_shape = balanced_x_train.shape[1:], return_sequences = True))
                    model.add(Dropout(0.2 * drop_out_multiplier))
                    model.add(BatchNormalization())

                    model.add(LSTM(representation_space, return_sequences = True))
                    model.add(Dropout(0.1 * drop_out_multiplier))
                    model.add(BatchNormalization())

                    model.add(LSTM(representation_space))
                    model.add(Dropout(0.2 * drop_out_multiplier))
                    model.add(BatchNormalization())

                    model.add(Dense(32, activation = 'relu'))
                    model.add(Dropout(0.2 * drop_out_multiplier))

                    model.add(Dense(2, activation = 'softmax'))

                    if optimizer == 'Adam':
                        opt = tf.keras.optimizers.Adam(lr=learning_rate, decay=1e-6)

                    else:
                        opt  = tf.keras.optimizers.SGD(learning_rate = 0.001, momentum = 0)



                    model.compile(
                        loss='sparse_categorical_crossentropy',
                        optimizer=opt,
                        metrics=['accuracy']
                    )

                    

                    tensorboard = TensorBoard(log_dir="logs\{}".format(NAME), profile_batch=0)

                    history = model.fit(
                        balanced_x_train, balanced_y_train,
                        batch_size=batch_size,
                        epochs=300,
                        validation_data=(balanced_x_test, balanced_y_test),
                        callbacks = [tensorboard],
                        verbose = 0
                    )
                    
                    score = model.evaluate(balanced_x_test, balanced_y_test, verbose=0)
                    
                    results[NAME] = score[1]
                    
                    #model.save("models\{}".format(NAME))
                    
                    counter += 1
                

1 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(5e-05)-batch_size(32)
-------------------------------------------
2 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(5e-05)-batch_size(64)
-------------------------------------------
3 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(5e-05)-batch_size(128)
-------------------------------------------
4 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(0.0001)-batch_size(32)
-------------------------------------------
5 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(0.0001)-batch_size(64)
-------------------------------------------
6 / 480 models
Model: representation_space(32)-drop_out_multiplier(0.5)-optimizer(Adam)-learning_rate(0.0001)-batch_size(128)
-------------------------------

50 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(5e-05)-batch_size(64)
-------------------------------------------
51 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(5e-05)-batch_size(128)
-------------------------------------------
52 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(0.0001)-batch_size(32)
-------------------------------------------
53 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(0.0001)-batch_size(64)
-------------------------------------------
54 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(0.0001)-batch_size(128)
-------------------------------------------
55 / 480 models
Model: representation_space(32)-drop_out_multiplier(1)-optimizer(Adam)-learning_rate(0.0005)-batch_size(32)
------------------------------------

99 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(5e-05)-batch_size(128)
-------------------------------------------
100 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(0.0001)-batch_size(32)
-------------------------------------------
101 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(0.0001)-batch_size(64)
-------------------------------------------
102 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(0.0001)-batch_size(128)
-------------------------------------------
103 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(0.0005)-batch_size(32)
-------------------------------------------
104 / 480 models
Model: representation_space(32)-drop_out_multiplier(2)-optimizer(Adam)-learning_rate(0.0005)-batch_size(64)
------------------------------

148 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0001)-batch_size(32)
-------------------------------------------
149 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0001)-batch_size(64)
-------------------------------------------
150 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0001)-batch_size(128)
-------------------------------------------
151 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0005)-batch_size(32)
-------------------------------------------
152 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0005)-batch_size(64)
-------------------------------------------
153 / 480 models
Model: representation_space(64)-drop_out_multiplier(0.75)-optimizer(Adam)-learning_rate(0.0005)-batch_size(128)
----------

Epoch 1/500
60/60 [==============================] - 8s 127ms/step - loss: 0.8701 - accuracy: 0.5147 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 2/500
60/60 [==============================] - 6s 103ms/step - loss: 0.8101 - accuracy: 0.5273 - val_loss: 0.7022 - val_accuracy: 0.5000
Epoch 3/500
60/60 [==============================] - 5s 77ms/step - loss: 0.7806 - accuracy: 0.5575 - val_loss: 0.7095 - val_accuracy: 0.5000
Epoch 4/500
60/60 [==============================] - 3s 55ms/step - loss: 0.7890 - accuracy: 0.5436 - val_loss: 0.7148 - val_accuracy: 0.5000
Epoch 5/500
60/60 [==============================] - 3s 56ms/step - loss: 0.7846 - accuracy: 0.5278 - val_loss: 0.7007 - val_accuracy: 0.5100
Epoch 6/500
60/60 [==============================] - 3s 58ms/step - loss: 0.7573 - accuracy: 0.5433 - val_loss: 0.6826 - val_accuracy: 0.5515
Epoch 7/500
60/60 [==============================] - 3s 55ms/step - loss: 0.7546 - accuracy: 0.5441 - val_loss: 0.6868 - val_accuracy: 0.5150
Epoc

Epoch 57/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6760 - accuracy: 0.5549 - val_loss: 0.6644 - val_accuracy: 0.5930
Epoch 58/500
60/60 [==============================] - 4s 68ms/step - loss: 0.6667 - accuracy: 0.5672 - val_loss: 0.6638 - val_accuracy: 0.6047
Epoch 59/500
60/60 [==============================] - 4s 66ms/step - loss: 0.6719 - accuracy: 0.5583 - val_loss: 0.6633 - val_accuracy: 0.6047
Epoch 60/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6654 - accuracy: 0.5643 - val_loss: 0.6586 - val_accuracy: 0.5947
Epoch 61/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6656 - accuracy: 0.5546 - val_loss: 0.6563 - val_accuracy: 0.6047
Epoch 62/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6651 - accuracy: 0.5656 - val_loss: 0.6818 - val_accuracy: 0.5764
Epoch 63/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6645 - accuracy: 0.5709 - val_loss: 0.6624 - val_accuracy: 0.6013

Epoch 171/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6481 - accuracy: 0.5719 - val_loss: 0.6727 - val_accuracy: 0.5648
Epoch 172/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6458 - accuracy: 0.5751 - val_loss: 0.6682 - val_accuracy: 0.5831
Epoch 173/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6505 - accuracy: 0.5682 - val_loss: 0.6690 - val_accuracy: 0.5731
Epoch 174/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6437 - accuracy: 0.5675 - val_loss: 0.6655 - val_accuracy: 0.5781
Epoch 175/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6435 - accuracy: 0.5711 - val_loss: 0.6701 - val_accuracy: 0.5664
Epoch 176/500
60/60 [==============================] - 4s 67ms/step - loss: 0.6494 - accuracy: 0.5622 - val_loss: 0.6642 - val_accuracy: 0.5764
Epoch 177/500
60/60 [==============================] - 4s 66ms/step - loss: 0.6490 - accuracy: 0.5659 - val_loss: 0.6685 - val_accuracy:

Epoch 228/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6450 - accuracy: 0.5735 - val_loss: 0.6783 - val_accuracy: 0.5631
Epoch 229/500
60/60 [==============================] - 4s 70ms/step - loss: 0.6466 - accuracy: 0.5609 - val_loss: 0.6880 - val_accuracy: 0.5498
Epoch 230/500
60/60 [==============================] - 5s 78ms/step - loss: 0.6431 - accuracy: 0.5887 - val_loss: 0.6950 - val_accuracy: 0.5515
Epoch 231/500
60/60 [==============================] - 4s 67ms/step - loss: 0.6454 - accuracy: 0.5822 - val_loss: 0.6790 - val_accuracy: 0.5565
Epoch 232/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6423 - accuracy: 0.5827 - val_loss: 0.6806 - val_accuracy: 0.5548
Epoch 233/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6414 - accuracy: 0.5806 - val_loss: 0.6815 - val_accuracy: 0.5249
Epoch 234/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6447 - accuracy: 0.5837 - val_loss: 0.6756 - val_accuracy:

Epoch 285/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6387 - accuracy: 0.5911 - val_loss: 0.6710 - val_accuracy: 0.5664
Epoch 286/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6400 - accuracy: 0.5824 - val_loss: 0.6749 - val_accuracy: 0.5581
Epoch 287/500
60/60 [==============================] - 4s 72ms/step - loss: 0.6421 - accuracy: 0.5856 - val_loss: 0.6739 - val_accuracy: 0.5631
Epoch 288/500
60/60 [==============================] - 4s 59ms/step - loss: 0.6422 - accuracy: 0.5806 - val_loss: 0.6846 - val_accuracy: 0.5548
Epoch 289/500
60/60 [==============================] - 3s 57ms/step - loss: 0.6421 - accuracy: 0.5777 - val_loss: 0.6915 - val_accuracy: 0.5515
Epoch 290/500
60/60 [==============================] - 3s 57ms/step - loss: 0.6385 - accuracy: 0.5913 - val_loss: 0.7003 - val_accuracy: 0.5349
Epoch 291/500
60/60 [==============================] - 3s 56ms/step - loss: 0.6416 - accuracy: 0.5832 - val_loss: 0.6806 - val_accuracy:

Epoch 342/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6369 - accuracy: 0.5995 - val_loss: 0.6844 - val_accuracy: 0.5598
Epoch 343/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6411 - accuracy: 0.5850 - val_loss: 0.6751 - val_accuracy: 0.5880
Epoch 344/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6372 - accuracy: 0.5992 - val_loss: 0.6770 - val_accuracy: 0.5831
Epoch 345/500
60/60 [==============================] - 5s 76ms/step - loss: 0.6385 - accuracy: 0.5929 - val_loss: 0.6772 - val_accuracy: 0.5814
Epoch 346/500
60/60 [==============================] - 4s 68ms/step - loss: 0.6392 - accuracy: 0.5966 - val_loss: 0.6812 - val_accuracy: 0.5615
Epoch 347/500
60/60 [==============================] - 4s 71ms/step - loss: 0.6414 - accuracy: 0.5832 - val_loss: 0.6780 - val_accuracy: 0.5631
Epoch 348/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6392 - accuracy: 0.5961 - val_loss: 0.6834 - val_accuracy:

Epoch 399/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6408 - accuracy: 0.5879 - val_loss: 0.6715 - val_accuracy: 0.5797
Epoch 400/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6359 - accuracy: 0.5953 - val_loss: 0.6755 - val_accuracy: 0.5814
Epoch 401/500
60/60 [==============================] - 4s 64ms/step - loss: 0.6380 - accuracy: 0.5892 - val_loss: 0.6840 - val_accuracy: 0.5648
Epoch 402/500
60/60 [==============================] - 5s 84ms/step - loss: 0.6403 - accuracy: 0.5890 - val_loss: 0.6732 - val_accuracy: 0.5797
Epoch 403/500
60/60 [==============================] - 4s 73ms/step - loss: 0.6393 - accuracy: 0.5850 - val_loss: 0.6765 - val_accuracy: 0.5648
Epoch 404/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6379 - accuracy: 0.5934 - val_loss: 0.6826 - val_accuracy: 0.5581
Epoch 405/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6392 - accuracy: 0.5892 - val_loss: 0.6862 - val_accuracy:

Epoch 456/500
60/60 [==============================] - 4s 66ms/step - loss: 0.6336 - accuracy: 0.5955 - val_loss: 0.6849 - val_accuracy: 0.5648
Epoch 457/500
60/60 [==============================] - 4s 68ms/step - loss: 0.6394 - accuracy: 0.5874 - val_loss: 0.6647 - val_accuracy: 0.5864
Epoch 458/500
60/60 [==============================] - 4s 71ms/step - loss: 0.6350 - accuracy: 0.5940 - val_loss: 0.6875 - val_accuracy: 0.5449
Epoch 459/500
60/60 [==============================] - 4s 63ms/step - loss: 0.6383 - accuracy: 0.5856 - val_loss: 0.6746 - val_accuracy: 0.5681
Epoch 460/500
60/60 [==============================] - 4s 62ms/step - loss: 0.6392 - accuracy: 0.5837 - val_loss: 0.6621 - val_accuracy: 0.6030
Epoch 461/500
60/60 [==============================] - 4s 65ms/step - loss: 0.6356 - accuracy: 0.6045 - val_loss: 0.6666 - val_accuracy: 0.5764
Epoch 462/500
60/60 [==============================] - 5s 83ms/step - loss: 0.6390 - accuracy: 0.5874 - val_loss: 0.6686 - val_accuracy:

KeyboardInterrupt: 

In [86]:
score = model.evaluate(balanced_x_train, balanced_y_train, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

120/120 [==============================] - 2s 14ms/step - loss: 0.6192 - accuracy: 0.6108
Test loss: 0.6192388534545898
Test accuracy: 0.6107611656188965


NameError: name 'balanced_x_train' is not defined